In [43]:
# ENCODER
import string
chars = string.ascii_letters + string.digits + string.punctuation
chars_nums = dict(zip(chars, range(len(chars))))
n_chars = len(chars)

def strToVec(str):
    return [chars_nums[i] for i in str], len(str)

# pad sequences and sort the tensor
def pad_sequences(vectorized_seqs, seq_lengths):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    return seq_tensor

# Create necessary variables, lengths, and target
def make_variables(names):
    sequence_and_length = [strToVec(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths)

In [49]:
# MODEL
import torch.nn as nn

class LSTMClassifier(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        # Note: we run this all at once (over the whole input sequence)

        # input = B x S . size(0) = B
        batch_size = input.size(0)

        # input:  B x S  -- (transpose) --> S x B
        input = input.t()

        # Embedding S x B -> S x B x I (embedding size)
        print("  input", input.size())
        embedded = self.embedding(input)
        print("  embedding", embedded.size())

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        output, hidden = self.gru(embedded, hidden)
        print("  gru hidden output", hidden.size())
        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden)
        print("  fc output", fc_output.size())
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return Variable(hidden)

In [111]:
# MODEL
import torch.nn as nn

class LSTMClassifier(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        # Note: we run this all at once (over the whole input sequence)

        # input = B x S . size(0) = B
        batch_size = input.size(0)

        # input:  B x S  -- (transpose) --> S x B
        input = input.t()

        # Embedding S x B -> S x B x I (embedding size)
        print("  input", input.size())
        embedded = self.embedding(input)
        print("  embedding", embedded.size())

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        output, (hidden, _) = self.lstm(embedded, hidden)
        print("  gru hidden output", hidden.size())
        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden)
        print("  fc output", fc_output.size())
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                  Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return hidden

In [119]:
import encoder as e
import torch
from torch.autograd import Variable
from data import TextLoader
data_loader = TextLoader('data/')
HIDDEN_SIZE = 100
N_CLASSES = len(data_loader.tag2id)
N_CHARS = len(data_loader.token_set)
classifier = LSTMClassifier(N_CHARS, HIDDEN_SIZE, N_CLASSES)
print(data_loader.test_data)
for name in names:
    arr = e.strToVec(name)
    inp = Variable(torch.LongTensor([arr]))
    out = classifier(inp)
    print("in", inp.size(), "out", out.size())


inputs = make_variables(names)
out = classifier(inputs)
print("batch in", inputs.size(), "batch out", out.size(), 'out ', out)

Data statistics: 
poly_fun 10000
square_fun 10000
lin_fun 10000
frac_lin_fun 10000
ration_fun 10000
cube_fun 10000
Train categories: ['cube_fun', 'frac_lin_fun', 'lin_fun', 'poly_fun', 'ration_fun', 'square_fun']
Dev categories: ['cube_fun', 'frac_lin_fun', 'lin_fun', 'poly_fun', 'ration_fun', 'square_fun']
Test categories: ['cube_fun', 'frac_lin_fun', 'lin_fun', 'poly_fun', 'ration_fun', 'square_fun']
[('$\\frac{242}{13}t^{3}+\\frac{4}{6}t^{2}-\\frac{11}{457}$', 'cube_fun'), ('$\\frac{4}{7}x^{10}+5,588x^{8}-x^{5}-x^{4}-x^{3}$', 'poly_fun'), ('$\\frac{158}{30}z^{3}+21,21z^{2}-2z$', 'cube_fun'), ('$w^{3}-\\frac{545}{317}w^{2}+3w-16,566$', 'cube_fun'), ('$\\frac{22,10273v-\\frac{58}{18}}{v}$', 'frac_lin_fun'), ('$-21,607623w^{2}+w-40$', 'square_fun'), ('$w^{2}-71w+3,75251$', 'square_fun'), ('$\\frac{z^{16}+z^{13}-36z^{5}}{\\frac{13}{142}z^{18}-z^{15}-z^{14}-z^{9}+29z^{7}-z^{4}-z}$', 'ration_fun'), ('$-w^{3}+11,147w^{2}-\\frac{4}{10}w-16,7656$', 'cube_fun'), ('$\\frac{w+2}{20,7w-6}$', 'fr

In [ ]:
import os
import sys
import argparse
import time
import random
import utils
import pdb

args = parser.parse_args()
print(args)
# data_loader = TextLoader(args.data_dir)

# train_data = data_loader.train_data
# dev_data = data_loader.dev_data
# test_data = data_loader.test_data

# char_vocab = data_loader.token2id
# tag_vocab = data_loader.tag2id
# char_vocab_size = len(char_vocab)